## Example - logical boundary condition test

This example tests the logical boundary condition by looking at the sheath heat transmission coefficient for a Maxwellian distribution

This is the v2.0.0 rewrite of the original test

In [ ]:
import numpy as np

import RMK_support as rmk
import RMK_support.common_models as cm
from RMK_support import node,varFromNode


### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27 #atomic mass unit
ionMass = 2.014*amu # deuterium mass
epsilon0 = 8.854188e-12 #vacuum permittivity 
heavySpeciesMass = 2.014 #in amus


### Context initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_lbc_test/" )

### Grid setup

In [ ]:
xGrid = np.ones(4) 
L = sum(xGrid)
dv0 = 0.05
cv = 1.025
vGrid = [dv0/2]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 1 
rk.grid = rmk.Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True)

### Variables

In [ ]:
nInit = 1.0 + 0.001*np.sin(2*np.pi*rk.grid.xGrid/L)
TInit = np.ones(rk.grid.numX)
fInit = np.zeros([rk.grid.numX,rk.grid.numH,rk.grid.numV])
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = (np.pi*TInit[i])**(-1.5) * nInit[i]* np.exp(-rk.grid.vGrid**2/TInit[i])

numerical_dens = rk.grid.velocityMoment(fInit,0,1)
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = nInit[i] *fInit[i,rk.grid.getH(0)-1,:]/numerical_dens[i]

f,f_dual = rmk.varAndDual("f",rk.grid,isDistribution=True,data=fInit)
n,n_dual = rmk.varAndDual("n",rk.grid,derivation=rk.textbook["densityMoment"],derivationArgs=["f"])

rk.variables.add(f,n)

nb_right = rmk.derivations.BoundedExtrapolationDerivation("nb_right",extrapolationType="linlog")(n)
nb_left = rmk.derivations.BoundedExtrapolationDerivation("nb_left",extrapolationType="linlog",leftBoundary=True)(n)

ionCurrent_right = varFromNode("ionCurrent_right",rk.grid,node=np.sqrt(elMass/ionMass)*node(nb_right),isScalar=True)

ionCurrent_left = varFromNode("ionCurrent_left",rk.grid,node=-np.sqrt(elMass/ionMass)*node(nb_left),isScalar=True)

rk.variables.add(nb_right,nb_left,ionCurrent_right,ionCurrent_left)

# Boundary property value containers

for varName in ["gamma_right","potential_right","temp_right","gamma_left","potential_left","temp_left"]:
    rk.variables.add(rmk.Variable(varName,rk.grid,isDerived=True,isScalar=True))

### LBC models

In [ ]:
LBCleft = cm.logicalBCModel(rk.grid,f,ionCurrent_left,n,n_dual,nb_left,leftBoundary=True,evolvedHarmonics=[1])

LBCright = cm.logicalBCModel(rk.grid,f,ionCurrent_right,n,n_dual,nb_right,evolvedHarmonics=[1])

rk.models.add(LBCleft,LBCright)

### Manipulators

Adding manipulators to extract boundary properties from modelbound data

In [ ]:
rk.manipulators.add(rmk.MBDataExtractor("gammaRight",LBCright,LBCright.mbData["gamma"],rk.variables["gamma_right"]))
rk.manipulators.add(rmk.MBDataExtractor("potentialRight",LBCright,LBCright.mbData["potential"],rk.variables["potential_right"]))
rk.manipulators.add(rmk.MBDataExtractor("tempRight",LBCright,LBCright.mbData["shTemp"],rk.variables["temp_right"]))

rk.manipulators.add(rmk.MBDataExtractor("gammaLeft",LBCleft,LBCleft.mbData["gamma"],rk.variables["gamma_left"]))
rk.manipulators.add(rmk.MBDataExtractor("potentialLeft",LBCleft,LBCleft.mbData["potential"],rk.variables["potential_left"]))
rk.manipulators.add(rmk.MBDataExtractor("tempLeft",LBCleft,LBCleft.mbData["shTemp"],rk.variables["temp_left"]))

### Integrator and timestep options

0 step to check analytic solution

In [ ]:
integrator = rmk.BDEIntegrator("BDE",absTol=10.0,convergenceVars=[f])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=0.0,steps=integrationStep) 
rk.integrationScheme.setFixedNumTimesteps(1,1) 


#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Logical boundary condition test")

### Create config file

In [ ]:
rk.writeConfigFile()


### Data analysis


#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadedData = rk.loadSimulation(onlySteps=[1])
dataset = loadedData.dataset

Compare solution to analytic value. Note that $\gamma_e$ saved as modelbound data is calculated using the cut-off distribution temperature, so to rescale to unit temperature the analytical gamma must be divided by the sheath temperature variable temp_right.

In [ ]:
analyticGamma = (2-0.5*np.log(4*np.pi*elMass/ionMass))/dataset["temp_right"].data[0]
abs(dataset["gamma_right"].data[0] - analyticGamma)/analyticGamma

In [ ]:
nalyticGamma = (2-0.5*np.log(4*np.pi*elMass/ionMass))/dataset["temp_left"].data[0]
abs(dataset["gamma_left"].data[0] - analyticGamma)/analyticGamma